In [1]:
import numpy as np
import pandas as pd
import Levenshtein

In [2]:
def leventstein_similarity(str1:str, str2:str):
    return 1 - Levenshtein.distance(str1, str2) / max(len(str1), len(str2))

In [3]:
def same_similarity(str1:str, str2:str):
    return str1==str2

In [26]:
def tokenize_date(date):
    # Tokenize the date string into year, month, and day
    return set(date.split('-'))

def date_similarity(date1, date2, threshold=1):
    
    if date1 is None or date2 is None:
        return 0
    
    tokens1 = tokenize_date(date1)
    tokens2 = tokenize_date(date2)
    
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    jaccard_similarity = intersection / union
    
    # if jaccard_similarity >= threshold:
    #     return True
    # else:
    #     return False
    return jaccard_similarity

date1 = "2000-09-14"
date2 = "2000-14-09"
threshold = 0.7  # You can adjust this threshold as needed

if date_similarity(date1, date2, threshold):
    print("Dates are similar.")
else:
    print("Dates are not similar.")

Dates are similar.


In [5]:
def get_similarity_matrix(similarity_function, col1, col2):

    similarity_matrix = np.zeros((len(col1), len(col2)))

    for i, entry1 in enumerate(col1):
        for j, entry2 in enumerate(col2):

            similarity_matrix[i, j] = similarity_function(entry1, entry2) 
        # print(f"Row {i}")     

    return similarity_matrix

In [6]:
def create_result(df1, df2, similarity_matrix, normalization=1):
    matches = pd.DataFrame()
    matches['df1'] = df1['id']
    matches['df2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['id'].values
    matches['name1'] = df1['name'].values
    matches['birthdate1'] = df1['birthdate'].values
    matches['name2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['name'].values
    matches['birthdate2'] = df2.iloc[similarity_matrix.argmax(axis=1)]['birthdate'].values
    matches['score'] = similarity_matrix.max(axis=1)/normalization
    return matches

In [7]:
df_eafc = pd.read_xml('../Football-Project/data/xml-files/eafc_final.xml')
df_fm23 = pd.read_xml('../Football-Project/data/xml-files/fm23_final.xml')
df_tm = pd.read_xml('../Football-Project/data/xml-files/tm_final.xml')

In [14]:
subset_size = 500
# df_eafc_sampled = df_eafc.sample(n=subset_size, random_state=42)
df_fm23_sampled = df_fm23.sample(n=subset_size, random_state=42)
# df_tm_sampled = df_tm.sample(n=subset_size, random_state=42)

In [27]:
similarity_matrix_name = get_similarity_matrix(leventstein_similarity, df_fm23_sampled['name'], df_tm['name'])
similarity_matrix_date = get_similarity_matrix(date_similarity, df_fm23_sampled['birthdate'], df_tm['birthdate'])

In [54]:
df_name = create_result(df_fm23_sampled, df_tm, similarity_matrix_name)
df_date = create_result(df_fm23_sampled, df_tm, similarity_matrix_date)
df = create_result(df_fm23_sampled, df_tm, (1.4*similarity_matrix_name) + (0.6*similarity_matrix_date), 2)
perfect_matches = df[df.score == 1]
perfect_matches["score"] = True
perfect_matches.to_csv('perfect_matches_fm_tm.csv', index=False)
print(len(df[df.score <= 0.50]))
print(len(perfect_matches))

201
150


C:\Users\555ka\AppData\Local\Temp\ipykernel_5176\1575917578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfect_matches["score"] = True


In [51]:
pd.set_option('display.max_rows', None)
display(df[df.score <= 0.60].sort_values(by=['score'], ascending=False))

,df1,df2,name1,birthdate1,name2,birthdate2,score
17024,fm_18024,tm_5478,Nordan Mukiele,2006-02-12,Nordi Mukiele,1997-11-01,0.600000
9425,fm_10425,tm_3161,Cui Wei,1994-08-16,Wei Cui,1994-08-16,0.600000
2830,fm_3830,tm_7557,Daniel Pérez,2002-01-17,Daniel Peretz,2000-07-10,0.592308
8116,fm_9116,tm_6509,Francesco Stazi,2005-10-10,Francesco Gelli,1996-10-15,0.588333
7112,fm_8112,tm_3943,Antonio Gasparro,2005-01-04,Antonio Nusa,2005-04-17,0.587500
16989,fm_17989,tm_1658,Gabriel Carvalho,2004-01-11,Gabriele Corbo,2000-01-11,0.587500
12905,fm_13905,tm_10050,Adam Watson,2006-02-22,Adam Wharton,2004-06-02,0.585000
16174,fm_17174,tm_7876,Dinis Rodrigues,2005-01-12,Dennis Seimen,2005-12-01,0.580000
6113,fm_7113,tm_8137,Andrea Errico,1999-01-01,Andreas Bruus,1999-01-16,0.576923
7623,fm_8623,tm_9158,Francesco Moratti,2006-02-07,Francesco Donati,2001-01-20,0.576471
